In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
#AnimalShelter Module
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
#username and password and CRUD Python module name

username = "aacuser"
password = "abc123"
shelter = AnimalShelter()


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())




app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('David Roehm CS-340'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    

  ##Pie Chart Function
    html.Div([
        html.Label(['Grazioso Salvare Animal Chart']),
        dcc.Dropdown(
            id='my_dropdown',
            options=[
                {'label':'Animal Type','value':'animal_type'},
                {'label':'Breed','value':'breed'},
                {'label':'Color','value':'color'}  
            ],
            value='Animal Type',
            multi=False,
            clearable=False,
            style={"width":"50%"}
        ),     
        
    ]),
    html.Div([
        dcc.Graph(id='the_graph')
    
    ]),

    ##Filter buttons
    html.Div(className='row',
             style={'display' : 'flex'},
                 children=[
                     html.Button(id='submit-button-one',n_clicks=0,children='Water Rescue'),
                     html.Button(id='submit-button-two',n_clicks=0,children='Mountain or Wilderness Rescue'),
                     html.Button(id='submit-button-three',n_clicks=0,children='Disaster Rescue or Individual Tracking'),
                     html.Button(id='submit-button-four',n_clicks=0,children='Reset')
                 ]),    
dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
          ##Datatable filter options
        filter_action='native',
        sort_action='native',
        editable = False,
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_size=15
    ),
html.Br(),
html.Hr(),
html.Div(
            id='map-id',
            className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
##Interactivity to buttons##
#@app.callback(Output('datatable-interactivity',"data"),
#             [Input('submit-button-one', 'n_clicks'),Input('submit-button-two','n_clicks'),
#              ])
#def on_click(bt1,bt2):
#    #start case
#    if (int(bt1) ==0 and int(bt2) ==0):
#        df = pd.DataFrame.from_records(shelter.read({}))
#    elif (int(bt1) > int(bt2)):
#        df = pd.DataFrame(list(shelter.read({"animal_type":"Cat"})))
#    elif (int(bt2) > int(bt1)):
#        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog"})))
#    return df.to_dict('records')


#-----Pie Chart Callback
@app.callback(
    Output('the_graph', 'figure'),
    [Input('my_dropdown', 'value')]
)
def update_piegraph(my_dropdown):
    dff=df
    
    piechart=px.pie(
        data_frame=dff,
        names=my_dropdown,
        hole=.15,
    )
    return (piechart)   


#------Pie Chart Callback

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
        } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#Geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
            dl.Marker(position=[30.75,-87.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
        
        
        
    ]

app